In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: Obtains the turnover/lodgement count and amount of a retail loan customer 12 months before the latest loan date
# In case customers have no lodgement in a month, that/those months are made zero before passing them in
# for average and standard deviation

In [19]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
import utils
from sqlalchemy import engine
import sqlalchemy
import dateutil.relativedelta
from sqlalchemy import types, create_engine

#set working directory
os.chdir(config.working_dir)

In [3]:
pd.set_option('display.max_columns', 500)

In [5]:
# db credentials authentication

In [4]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Enter username: data_analyst
Enter Password:········
Press 'y' to continue, 'c' to cancel or escape:y
Testing Connection...
Incorrect credentials. Please Try again!
Enter username: data_analyst
Enter Password:········
Press 'y' to continue, 'c' to cancel or escape:y
Testing Connection...
Connection Succesful!


In [21]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

#### Retreive info previously extracted

In [45]:
df_retail_loan_cust_info = pd.read_csv('df_retail_loan_cust_info.csv')
df_retail_loan_cust = pd.read_csv('df_retail_loan_cust.csv')
df_retail_loan_cust_acc_info = pd.read_csv('df_retail_loan_cust_acc_info.csv')
df_retail_loan_cust_turn_lodge_count_info = pd.read_csv('df_retail_loan_cust_turn_lodge_count_info.csv')
df_digital_trxns_info = pd.read_csv('df_digital_trxns_info.csv')

In [ ]:
# convert joining columns to numbers/integers

In [52]:
df_retail_loan_cust_info[['BRA_CODE','CUS_NUM']]=df_retail_loan_cust_info[['BRA_CODE','CUS_NUM']].astype(int)
df_retail_loan_cust[['BRA_CODE','CUS_NUM']]=df_retail_loan_cust[['BRA_CODE','CUS_NUM']].astype(int)
df_retail_loan_cust_acc_info[['BRA_CODE','CUS_NUM']]=df_retail_loan_cust_acc_info[['BRA_CODE','CUS_NUM']].astype(int)
df_retail_loan_cust_turn_lodge_count_info[['BRA_CODE','CUS_NUM']]=df_retail_loan_cust_turn_lodge_count_info[['BRA_CODE','CUS_NUM']].astype(int)
df_digital_trxns_info[['BRA_CODE','CUS_NUM']]=df_digital_trxns_info[['BRA_CODE','CUS_NUM']].astype(int)

In [ ]:
# join data pieces together

In [53]:
df_consolidated_retail_loan = df_retail_loan_cust.merge(
    df_retail_loan_cust_acc_info, how='left', on=['BRA_CODE','CUS_NUM']).merge(
    df_retail_loan_cust_turn_lodge_count_info, how='left', on=['BRA_CODE','CUS_NUM']).merge(
    df_digital_trxns_info, how='left', on=['BRA_CODE','CUS_NUM'])

In [54]:
df_consolidated_retail_loan = df_consolidated_retail_loan.fillna(0)

In [55]:
df_consolidated_retail_loan = df_consolidated_retail_loan.merge(
    df_retail_loan_cust_info, how='left', on=['BRA_CODE','CUS_NUM'])

In [56]:
df_consolidated_retail_loan['DATE_OPEN']=pd.to_datetime(df_consolidated_retail_loan['DATE_OPEN'])
df_consolidated_retail_loan['CUS_DATE_OPEN']=pd.to_datetime(df_consolidated_retail_loan['CUS_DATE_OPEN'])
df_consolidated_retail_loan['BIR_DATE']=pd.to_datetime(df_consolidated_retail_loan['BIR_DATE'])

In [57]:
df_consolidated_retail_loan.to_csv('df_consolidated_retail_loan.csv', index=False)

In [58]:
conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))
df_consolidated_retail_loan.to_sql('cs_consolidated_retail_loan', conn, index=False,
                                   if_exists='replace', chunksize=10000, dtype=utils.sqlcol(df_consolidated_retail_loan))

In [51]:
# df_consolidated_retail_loan.info()